# Reminder: Download the latest 'data' folder from Teams and replace yours before start working.

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [2]:
# Step 1: Load all raw data files
# We read multiple data sources including influent, oxygen sensors A and B, ammonium, nitrate, phosphate,
# effluent, and weather data from various formats (parquet, excel, csv).
df_influent = pd.read_parquet('data/raw-data/Influent_2023.parquet')
df_oxygen_A = pd.read_parquet('data/raw-data/oxygen_a_2023.parquet')
df_oxygen_B = pd.read_parquet('data/raw-data/oxygen_b_2023.parquet')
df_ammonium = pd.read_parquet('data/raw-data/ammonium_2023.parquet')
df_nitrite = pd.read_parquet('data/raw-data/nitrate_2023.parquet')
df_phosphate = pd.read_parquet('data/raw-data/phosphate_2023.parquet')
df_effluent = pd.read_excel('data/raw-data/effluent_2023.xlsx')
df_weather = pd.read_csv('data/weather-data/weather_2023.csv', sep=',')

In [3]:
# Check the year of these dataframes are 2023, and see their columns names & records frenquency.

# influent data - recorded every minute
# df_influent.head()

# oxygen data (A & B) - need to be merged (take the average) first - recorded every minute
# df_oxygen_A.head()
# df_oxygen_B.head()

# chemical data (ammonium, nitrite, phosphate) - recorded every minute
# df_ammonium.head()
# df_nitrite.head()
# df_phosphate.head()

# effluent data - recorded every 15 minutes
# df_effluent.head() 15min

# weather data - recorded every hour
# df_weather.head()

In [4]:
# Check for duplicate timestamps in oxygen sensor A data to understand data quality issues
duplicates = df_oxygen_A[df_oxygen_A.duplicated(subset='datumBeginMeting', keep=False)]
print(duplicates.sort_values('datumBeginMeting').head(20))

            historianTagnummer hstWaarde     datumBeginMeting  \
40439  EDE_B121069901_K600.MTW     1.807  2023-10-29 01:59:00   
40440  EDE_B121069901_K600.MTW     1.893  2023-10-29 01:59:00   
40441  EDE_B121069901_K600.MTW     1.835  2023-10-29 02:00:00   
40442  EDE_B121069901_K600.MTW     1.831  2023-10-29 02:00:00   
40443  EDE_B121069901_K600.MTW     1.824  2023-10-29 02:01:00   
40444  EDE_B121069901_K600.MTW     1.839  2023-10-29 02:01:00   
40445  EDE_B121069901_K600.MTW     1.867  2023-10-29 02:02:00   
40446  EDE_B121069901_K600.MTW     1.855  2023-10-29 02:02:00   
40447  EDE_B121069901_K600.MTW     1.845  2023-10-29 02:03:00   
40448  EDE_B121069901_K600.MTW     1.841  2023-10-29 02:03:00   
40449  EDE_B121069901_K600.MTW     1.863  2023-10-29 02:04:00   
40450  EDE_B121069901_K600.MTW     1.825  2023-10-29 02:04:00   
40451  EDE_B121069901_K600.MTW     1.874  2023-10-29 02:05:00   
40452  EDE_B121069901_K600.MTW     1.812  2023-10-29 02:05:00   
40454  EDE_B121069901_K60

In [5]:
# Define a function to process data with duplicates and missing timestamps
# This function:
# - Handles duplicate timestamps by averaging their values
# - Converts timestamps to Amsterdam timezone considering daylight saving time
# - Fills in missing timestamps by reindexing and forward filling missing values

def process_data_with_duplicates(df, time_col='datumBeginMeting', value_col='hstWaarde', freq='min'):
    # Make a copy to avoid modifying the original DataFrame
    df = df.copy()

    # Convert the time column to datetime format
    df[time_col] = pd.to_datetime(df[time_col])

    # Ensure the value column is numeric, coercing errors to NaN
    df[value_col] = pd.to_numeric(df[value_col], errors='coerce')

    # Find and handle duplicate timestamps by averaging their values
    duplicates = df.duplicated(subset=time_col, keep=False).sum()
    print(f"Found {duplicates} duplicate timestamps")
    if duplicates > 0:
        df = df.groupby(time_col, as_index=False)[value_col].mean()

    # Set the time column as index
    df.set_index(time_col, inplace=True)

    # Assume original timestamps are UTC and convert to Amsterdam time zone (handles DST automatically)
    df.index = df.index.tz_localize('UTC').tz_convert('Europe/Amsterdam')

    # Generate a complete timestamp range with specified frequency in Amsterdam time zone
    start = df.index.min()
    end = df.index.max()
    full_range = pd.date_range(start=start, end=end, freq=freq, tz='Europe/Amsterdam')

    # Reindex the DataFrame to the full range, marking missing timestamps as NaN
    df = df.reindex(full_range)

    # Remove timezone information, keeping naive timestamps
    df.index = df.index.tz_localize(None)

    # Report missing timestamps before filling
    missing_before = df[value_col].isna().sum()
    print(f"Initial missing timestamps count: {missing_before}")
    if missing_before > 0:
        print("Example missing timestamps:", df[df[value_col].isna()].index[:5])

    # Forward fill missing values to maintain continuity
    df.ffill(inplace=True)

    # Report remaining missing timestamps after filling
    missing_after = df[value_col].isna().sum()
    print(f"Missing timestamps after forward fill: {missing_after}")

    # Reset index to convert time back to a column
    df = df.reset_index()
    df = df.rename(columns={'index': time_col})

    return df

In [6]:
def process_oxygen_data(df_a, df_b):
    # Process sensor A data
    df_a_processed = process_data_with_duplicates(df_a, value_col='hstWaarde')
    df_a_processed = df_a_processed.rename(columns={'hstWaarde': 'oxygen_a'})
    
    # Process sensor B data
    df_b_processed = process_data_with_duplicates(df_b, value_col='hstWaarde')
    df_b_processed = df_b_processed.rename(columns={'hstWaarde': 'oxygen_b'})
    
    # Merge sensor A and B data on timestamp
    df_oxygen = pd.merge(df_a_processed, df_b_processed, on='datumBeginMeting', how='inner')
    
    # Calculate average oxygen value from both sensors
    df_oxygen['oxygen_avg'] = (df_oxygen['oxygen_a'] + df_oxygen['oxygen_b']) / 2
    
    return df_oxygen


In [7]:
# Apply processing to oxygen A and B data, printing status along the way
print("Processing oxygen sensor A data:")
df_oxygen_A_processed = process_data_with_duplicates(df_oxygen_A)
print("\nProcessing oxygen sensor B data:")
df_oxygen_B_processed = process_data_with_duplicates(df_oxygen_B)

Processing oxygen sensor A data:
Found 120 duplicate timestamps
Initial missing timestamps count: 611
Example missing timestamps: DatetimeIndex(['2023-01-03 10:09:00', '2023-01-03 10:10:00',
               '2023-01-03 10:11:00', '2023-01-03 10:12:00',
               '2023-01-03 10:13:00'],
              dtype='datetime64[ns]', freq=None)
Missing timestamps after forward fill: 0

Processing oxygen sensor B data:
Found 120 duplicate timestamps
Initial missing timestamps count: 611
Example missing timestamps: DatetimeIndex(['2023-01-03 10:09:00', '2023-01-03 10:10:00',
               '2023-01-03 10:11:00', '2023-01-03 10:12:00',
               '2023-01-03 10:13:00'],
              dtype='datetime64[ns]', freq=None)
Missing timestamps after forward fill: 0


In [8]:
# Combine processed oxygen sensor data and calculate average oxygen concentration
a_clean = df_oxygen_A_processed.rename(columns={'hstWaarde': 'oxygen_a'})
b_clean = df_oxygen_B_processed.rename(columns={'hstWaarde': 'oxygen_b'})
df_oxygen = pd.merge(a_clean, b_clean, on='datumBeginMeting', how='inner')
df_oxygen['oxygen_avg'] = (df_oxygen['oxygen_a'] + df_oxygen['oxygen_b']) / 2

In [9]:
# Process other water quality datasets: influent, ammonium, nitrate, and phosphate
# We apply the same cleaning function to handle duplicates and missing timestamps
print("\nProcessing influent data:")
df_influent_processed = process_data_with_duplicates(df_influent)

print("\nProcessing ammonium data:")
df_ammonium_processed = process_data_with_duplicates(df_ammonium)

print("\nProcessing nitrate data:")
df_nitrate_processed = process_data_with_duplicates(df_nitrite)

print("\nProcessing phosphate data:")
df_phosphate_processed = process_data_with_duplicates(df_phosphate)


Processing influent data:
Found 120 duplicate timestamps
Initial missing timestamps count: 421
Example missing timestamps: DatetimeIndex(['2023-03-01 10:05:00', '2023-03-01 10:06:00',
               '2023-03-01 10:07:00', '2023-03-01 10:08:00',
               '2023-03-01 10:09:00'],
              dtype='datetime64[ns]', freq=None)
Missing timestamps after forward fill: 0

Processing ammonium data:
Found 120 duplicate timestamps
Initial missing timestamps count: 611
Example missing timestamps: DatetimeIndex(['2023-01-03 10:09:00', '2023-01-03 10:10:00',
               '2023-01-03 10:11:00', '2023-01-03 10:12:00',
               '2023-01-03 10:13:00'],
              dtype='datetime64[ns]', freq=None)
Missing timestamps after forward fill: 0

Processing nitrate data:
Found 120 duplicate timestamps
Initial missing timestamps count: 611
Example missing timestamps: DatetimeIndex(['2023-01-03 10:09:00', '2023-01-03 10:10:00',
               '2023-01-03 10:11:00', '2023-01-03 10:12:00',
     

In [10]:
# Merge all cleaned water quality data into a single DataFrame

# Rename columns for clarity before merging on timestamp
influent_clean = df_influent_processed.rename(columns={'hstWaarde': 'Influent'})
oxygen_clean = df_oxygen[['datumBeginMeting', 'oxygen_avg']].rename(columns={'oxygen_avg': 'Oxygen'})
ammonium_clean = df_ammonium_processed.rename(columns={'hstWaarde': 'Ammonium'})
nitrate_clean = df_nitrate_processed.rename(columns={'hstWaarde': 'Nitrate'})
phosphate_clean = df_phosphate_processed.rename(columns={'hstWaarde': 'Phosphate'})

# Merge all water quality datasets on the common timestamp column
df_water = influent_clean.merge(oxygen_clean, on='datumBeginMeting', how='inner') \
                        .merge(ammonium_clean, on='datumBeginMeting', how='inner') \
                        .merge(nitrate_clean, on='datumBeginMeting', how='inner') \
                        .merge(phosphate_clean, on='datumBeginMeting', how='inner')

After reviewing the weather dataset, I found that the Rain column only contains binary values: 0 indicates no rain, and 1 indicates rain. So, I decided to use the PrecipitationAmount column this time, which records the actual hourly precipitation amount in millimeters (mm). Additionally, the temperature values in the dataset have already been adjusted by dividing them by 10, so the unit is now degrees Celsius (°C).

In [11]:
# Process weather data
# Convert timestamp to Amsterdam timezone and rename columns for consistency
df_weather['Timestamp'] = pd.to_datetime(df_weather['Timestamp']).dt.tz_localize('UTC').dt.tz_convert('Europe/Amsterdam')
df_weather = df_weather.rename(columns={'Timestamp': 'date', 'PrecipitationAmount': 'Rainfall'})

In [12]:
# Merge water quality and weather datasets on datetime

# Remove timezone info for both datasets to ensure proper merging
df_water['date'] = pd.to_datetime(df_water['datumBeginMeting']).dt.tz_localize(None)
df_weather['date'] = pd.to_datetime(df_weather['date']).dt.tz_localize(None)

# Merge on 'date' column
df_main = pd.merge(df_water, df_weather, on='date', how='inner')

In [13]:
# Resample the merged data to hourly frequency by taking the mean
# This reduces data granularity from minutes to hours, which smooths noise and aligns with analysis needs
df_main.set_index('date', inplace=True)
df_main = df_main.resample('H').mean().reset_index()

/var/folders/6m/nmdc67354c9f77107p22bnd00000gn/T/ipykernel_62882/3977377120.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_main = df_main.resample('H').mean().reset_index()


In [14]:
# Final check for missing values in the combined dataset
print("\nFinal missing value check:")
print(df_main.isna().sum())


Final missing value check:
date                0
datumBeginMeting    1
Influent            1
Oxygen              1
Ammonium            1
Nitrate             1
Phosphate           1
Rainfall            1
Temperature         1
dtype: int64


In [15]:
# Display rows with missing values for review
missing_hours = df_main[df_main.isna().any(axis=1)]
print(missing_hours)

                    date datumBeginMeting  Influent  Oxygen  Ammonium  \
2017 2023-03-26 02:00:00              NaT       NaN     NaN       NaN   

      Nitrate  Phosphate  Rainfall  Temperature  
2017      NaN        NaN       NaN          NaN  


In [16]:
# Fill remaining missing values using forward fill to maintain data continuity
df_main.ffill(inplace=True)

In [17]:
# Save the cleaned and combined dataset to CSV for future use
df_main.to_csv("data/main_2023.csv", index=False)

# Display basic info and first few rows to confirm successful processing
df_main.info()
df_main.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              8760 non-null   datetime64[ns]
 1   datumBeginMeting  8760 non-null   datetime64[ns]
 2   Influent          8760 non-null   float64       
 3   Oxygen            8760 non-null   float64       
 4   Ammonium          8760 non-null   float64       
 5   Nitrate           8760 non-null   float64       
 6   Phosphate         8760 non-null   float64       
 7   Rainfall          8760 non-null   float64       
 8   Temperature       8760 non-null   float64       
dtypes: datetime64[ns](2), float64(7)
memory usage: 616.1 KB


,date,datumBeginMeting,Influent,Oxygen,Ammonium,Nitrate,Phosphate,Rainfall,Temperature
0,2023-01-01 01:00:00,2023-01-01 01:00:00,3330.199,1.2200,1.310,4.730,0.001,0.0,15.3
1,2023-01-01 02:00:00,2023-01-01 02:00:00,2916.007,0.0540,1.119,3.616,0.010,0.0,14.6
2,2023-01-01 03:00:00,2023-01-01 03:00:00,2573.291,1.5320,2.481,3.239,0.367,0.0,15.0
3,2023-01-01 04:00:00,2023-01-01 04:00:00,2624.023,1.7755,1.102,3.696,0.012,0.0,14.7
4,2023-01-01 05:00:00,2023-01-01 05:00:00,1426.117,0.1885,1.485,2.513,0.281,0.0,14.1


In [18]:
# Normalize all feature columns using Min-Max scaling

# only select numeric columns for scaling
df_features = df_main.select_dtypes(include=['number'])

# Initialize Min-Max scaler
scaler = MinMaxScaler()

# Fit scaler and transform the features
df_scaled_values = scaler.fit_transform(df_features)

# Create a DataFrame with scaled values and original column names
df_minmax_scaled = pd.DataFrame(df_scaled_values, columns=df_features.columns)

# Add back the date column
df_minmax_scaled['date'] = df_main['date']

# Reorder columns to keep date as the first column
df_minmax_scaled = df_minmax_scaled[['date'] + df_features.columns.tolist()]

# Save the scaled dataset to CSV
df_minmax_scaled.to_csv("data/minmax_scaled_main_2023.csv", index=False)

print("\nMin-Max scaling completed and saved!")


Min-Max scaling completed and saved!


In [19]:
# only select numeric columns for standardization
df_features = df_main.select_dtypes(include=['number'])

# Initialize the StandardScaler
scaler = StandardScaler()

# Apply standardization
df_standartization_scaled_values = scaler.fit_transform(df_features)

# Create a DataFrame with standardized values
df_standartization_scaled = pd.DataFrame(df_standartization_scaled_values, columns=df_features.columns)

# Add the date column back
df_standartization_scaled['date'] = df_main['date']

# Reorder columns to keep date first
df_standartization_scaled = df_standartization_scaled[['date'] + df_features.columns.tolist()]

# Save the standardized data to CSV
df_standartization_scaled.to_csv("data/standard_scaled_main_2023.csv", index=False)

print("\nStandardization scaling completed and saved!")



Standardization scaling completed and saved!
